## Exploiting the results

Discovery and exploition of the generated results using STAC  

In [ ]:
from urllib.parse import urlparse
import requests
from pystac import STAC_IO
from requests.auth import HTTPBasicAuth
from pystac import *
import matplotlib.pyplot as plt
import getpass 
import gdal

In [ ]:
%matplotlib inline

In [ ]:
username = 'eoepca-demo-storage'

In [ ]:
password = getpass.getpass('Storage password:')

In [ ]:
stac_catalog_endpoint = 'https://nx10438.your-storageshare.de/remote.php/dav/files/eoepca-demo-storage/wf-30d3c924-13b3-11eb-834e-0a580a8105b1/stac-results/catalog.json'

In [ ]:
def my_read_method(uri):
    
    parsed = urlparse(uri)
    
    if parsed.scheme.startswith('http'):
        
        return requests.get(uri, 
                            auth=HTTPBasicAuth(username, password)
                           ).text
    else:
        return STAC_IO.default_read_text_method(uri)

STAC_IO.read_text_method = my_read_method

In [ ]:
catalog = Catalog.from_file(stac_catalog_endpoint)

catalog.describe()

In [ ]:
item = next(catalog.get_items())

In [ ]:
item.get_assets()

In [ ]:
nbr_asset = item.get_assets()['nbr']

nbr_asset.get_absolute_href()

In [ ]:
def get_vsi_url(uri, username=None, password=None):
    
    
    parsed_url = urlparse(uri)

    if username is not None:
        
        url = '/vsicurl/{}://{}:{}@{}/{}'.format(list(parsed_url)[0],
                                                username, 
                                                password, 
                                                list(parsed_url)[1],
                                                list(parsed_url)[2])
    
    else:
        
        url = '/vsicurl/{}://{}/{}'.format(list(parsed_url)[0],
                                              list(parsed_url)[1],
                                              list(parsed_url)[2])
    
    return url 

In [ ]:
ds = gdal.Open(get_vsi_url(uri=nbr_asset.get_absolute_href(),
               username=username, 
               password=password))

In [ ]:
nbr_band = ds.GetRasterBand(1)

In [ ]:
nbr_data = nbr_band.ReadAsArray()

In [ ]:
fig = plt.figure(figsize=(25,25))

In [ ]:
imgplot = plt.imshow(nbr_data.reshape(nbr_data.shape[0],
                                      nbr_data.shape[1]), 
                     cmap=plt.cm.gray) 